In [21]:
import string
import math as m
import itertools as iTools
from nupack import *

#defining the toehold switch complex analysis model
thsModel = Model(material = "rna", celsius = 37)

#Filse for storing output
outputFile = open("triggerComplex.txt", "w")
outputFile2 = open("toeholdSwitch.txt", "w")

#Dictionary for storing user input
miRNADict = {}

#dictionary defining complimentary base pairing (no wobble pairing present)
baseDict = {"A":"U", "C":"G"}

#List of bases
baseList = ["A", "U", "C", "G"]

#list of and regions
andRegions = ("AAAAAA", "UUUUUU", "CCCCCC", "GGGGGG")

#complex structure in DU+
complexStructure = []

#strands to be put together in the file
strands = []

#list of domain names for NUPACK
domains = list((string.ascii_lowercase))

#list of complex.seq
seq = []

#number of iterations
iterCount = 0

#computes complimentary strand
def compStrand(rnaStrand):
    compliment = ""
    for i in rnaStrand:
        for key, value in baseDict.items():
            if i == key:
                compliment += value
            elif i == value:
                compliment += key
    return compliment

#reverses string
def reverse(rnaStrand):
  return rnaStrand[::-1]

#splits each miRNAInput into thirds and adds those to the storageArray
def splitString(miRNAInput):
    storageArray = []
    miRNAInput = miRNAInput.replace(" ","")
    n = round(len(miRNAInput)//3)
    part1 = miRNAInput[:n]
    part2 = miRNAInput[2*n+1:]
    part3 = miRNAInput[n:2*n+1]#modified
    storageArray.append(compStrand(part1))
    storageArray.append(compStrand(part2))
    storageArray.append(part3)#Modified
    return storageArray

#computes and gate
def andGateGen(sequenceDict):
    storageDict = sequenceDict.values()
    storageArray = []
    miRNAPartArray = []
    global unpairedMiddleArray
    unpairedMiddleArray = []#modified
 
    #splits the input strands into the required components for genetic switch
    for element in storageDict:
        storageArray = splitString(element)
        miRNAPartArray.append(storageArray[0])
        miRNAPartArray.append(storageArray[1])
        unpairedMiddleArray.append(storageArray[2])#modified

    #makes sure code runs on the end of 1 strand and the beginning of the other
    del miRNAPartArray[0]
    del miRNAPartArray[len(miRNAPartArray)-1]
    middleIndex = 0
    for element in miRNAPartArray:

        if(miRNAPartArray.index(element) % 2==0):
            #index is incrimented later to update the andRegion such that its adjacent base doesn't match itself
            index = 0

            #this sets the default value of the andRegion to AAAAAA
            andRegion = andRegions[index]
            
            
            #these are the parts of the and gates that bind to the toehold switch, specifically those complimentary to the miRNA
            global miRNA1, miRNA2

            miRNA1 = reverse(miRNAPartArray[miRNAPartArray.index(element)])

            miRNA2 = miRNAPartArray[miRNAPartArray.index(element) + 1]  
           
            #checking to see if the andRegion borders an identical base and changing it to correct this by incrimenting the index
            #this only needs to be done twice as in the "worst case scenario", ALL of the below code will run, making the anndRegion different to BOTH of its neighboring bases
            if (miRNA1[-1] == andRegion[1]):
                index += 1
                andRegion = andRegions[index]
                print(andRegion)
            if (reverse(miRNA2)[0] == andRegion[1]):
                index += 1
                andRegion = andRegions[index]
                print(andRegion)
            if (reverse(miRNA2)[0] and miRNA1[-1] == andRegion[1]):
                index += 1
                andRegion = andRegions[index]  
                print(andRegion)

            #defining a repeating unit for the complex
            duplex1 = "D" + str(len(miRNA1)) + "+"
            complexAnd = "U" + str(len(andRegion))
            duplex2 = "D" + str(len(miRNA2)) + "+"
            unpairedMiddle = "U" + str(len(unpairedMiddleArray[middleIndex]))#modified

            repeatUnit = [duplex1, complexAnd, duplex2, unpairedMiddle]      
           
           
            andGateDict = {}
            andGateArray = []
            # for gate in andRegions:
            #     if (reverse(miRNA2)[0] and miRNA1[-1] == gate[1]):
            #         energy = structure_energy(strands = [gate, compStrand(gate)],
            #                                  structure = "D6+", model = thsModel)
            #         andGateArray.append(energy)
            #         andGateDict[energy] = gate
            # andGateArray = sorted(andGateArray)
            # andRegion = andGateDict[andGateArray[0]]
           
            #construcing and gate out of end of 1 miRNA< start of the next one and the AND region in between consisting of 6 repeated bases
            andGate = miRNA1 + andRegion + reverse(miRNA2)
            
            #strands array contains UNORDERED strands, starting with and gates, there will always be 1 fewer and gates than miRNA
            strands.append(andGate)
            middleIndex+=1#modified

            #adding repeat unit to an empty array
            complexStructure.extend(repeatUnit)
        else:
            continue
    print("Done")

#Ensures correct inputs
def checkCorrectCharacters(characterList, userInput:string):
    userInput = userInput.replace(" ","")
    for element in userInput:
        if(characterList.count(element)==0):
            break
        else:
            return True
    return False

#gets the length of the unpaired start and end.
def unpairedParts(numMiRNA):
    global unpairedStartLength
    unpairedStartLength = len(list(miRNADict.values())[0]) - round((len(list(miRNADict.values())[0])//3))

    #this was the annoying thing I was talking about ages ago where due to divisibility by 3 resulting in a number that may round up or down depending on the remainder. fixed by brute force
    if (unpairedStartLength % 3 == 2):
        unpairedStartLength = unpairedStartLength + 1
    elif (unpairedStartLength % 3 == 1):
        unpairedStartLength = unpairedStartLength - 1
    else:
        unpairedStartLength = unpairedStartLength

    #converting unpairedStart and End to DU+ notation and adding to the beginning and end of complexStructure respectively
    if(numMiRNA == 1):
        unpairedStart = "U" + str(unpairedStartLength)
        complexStructure.insert(0, unpairedStart)
    else:
        unpairedStart = "U" + str(unpairedStartLength)
        unpairedEnd = "U" + str(len(list(miRNADict.values())[-1]) - len(miRNA2))
        complexStructure[-1] = unpairedEnd
        complexStructure.insert(0, unpairedStart)

#Combines two lists together into one array so that their constituent variables appear in an alternating sequence (miRNA, AND, miRNA, AND.... miRNA)
def listAddition(array1, array2):
   
    # integer = 0
    # while (integer < len(array1)):
    #     for domain in array2:
    #         if((array2.index(domain)%2 ==0) and (array2.index(domain) + 1 < len(array2))):
    #             array2.insert(array2.index(domain) + 1, array1[integer])
    #             integer += 1
    #         else:
    #             continue
   
    #dummy element makes the length of the list divisible by 2
    array1.append("DUMMY")
    # array2 = [sub[item] for item in range(len(array1)) for sub in [array2, array1]]
    array2 = [item for sublist in zip(array2, array1) for item in sublist]
    del array2[-1]
    del array1[-1]
   
    return array2
   

def domainOutputs():
    domainCodes = []

    #this writes the text to be put in the txt file for assigning domains to the DU+ structure, eg
    #domain a = AUCGACGACUGCCGUA
    #domain b = UUCGUAGCUAGCUGA etc...
    for strand, domain in zip(strands, domains):
        domainCode = "domain " + domain + " = " + strand
        domainCodes.append(domainCode)

    #this takes letters of the alphabet in order until there is 1 for each miRNA and AND gate
    global domainsUsed
    domainsUsed = domains[0:len(domainCodes)]

    #the domainsUsed list is split into 2 halves, one with letters to be assigned to miRNAs and the other to AND gates
    mid = int((len(domainsUsed) + 1)/2)
    miRNAs = list(domainsUsed[mid-1:])
    andGates = list(domainsUsed[0:mid-1])

    seq.extend(miRNAs)

    #reordering the miRNAs and AND gates to be in a pattern of: miRNA, AND gate, miRNA, AND gate, miRNA etc...
    print("before: ", seq)
    print(listAddition(andGates, seq))
    seq.extend(listAddition(andGates, seq))
    del seq[0:len(andGates)+1]
    print("after: ", seq)

    #writes all the calculated info to a text file
    outputFile.write(f'# miRNA complex structure in DU+ notation \nstructure complex = {"".join(complexStructure)}\n')
    outputFile.write(f'\n# base sequences for each RNA strand')
    for code in domainCodes:
        outputFile.write(f"\n" + code)
    outputFile.write(f'\n \n# mapping RNA strands to the complex')
    outputFile.write(f'\ncomplex.seq = ')
    global computedComplexStructure
    computedComplexStructure = complexStructure
    DUPlusToDPPlus(computedComplexStructure)
    for sequence in seq:
        outputFile.write(sequence + " ")

def toeholdSwitchGen(numMiRNA):
    #handles creatoion of toehold switch
    #assigning start and end of toeholdRegion
    orderedStrands = listAddition(strands[0:int((len(strands)-1)/2)], strands[int((len(strands)-1)/2):int(len(strands))])

    #this is the unpaired region of the toehold switch where the miRNA complex binds
    toehold = round((len(list(miRNADict.values())[0])//2))
   
    #this is the unpaired part of the toehold switch the miRNA, AND GATE complex would first bind to before the switch unzips
    toeholdRegion = []
   
    if(numMiRNA !=1):
        toeholdStart = list(miRNADict.values())[0][0:unpairedStartLength]
        toeholdEnd = list(miRNADict.values())[-1][len(miRNA2):]
       
        print(orderedStrands)
        thsAnds = []
        thsUnpaired = []
       
        for i,strand in enumerate(orderedStrands):
            #determining AND gates in the strands list, now ordered with listAddition, then determining what AND sections are used, eg AAAAAA, GGGGGG, UUUUUU, CCCCCC and adding them to another list
            if (i % 2 == 1):
                print(strand, " is an and gate")
                for j in andRegions:
                    if (j in strand):
                        thsAnd = j
                        thsAnds.append(thsAnd)
                        if (i == 1):
                            print("first and gate")
                        elif (i == len(orderedStrands) - 2):
                            print("last and gate")
                        else:
                            print("middle and gate")
                        print(thsAnds)
                        continue
            
            #if it's not an and gate, it must be and miRNA, a distinction is made between first and final miRNA in ordered strand complexes as they will have different armound of bases paired with the switch when compared to middle miRNA
            else:
                print(strand, " isn't an and gate")
                if (i == 0):
                    print("first miRNA")
                    for j in andRegions:
                        if j in (orderedStrands[i+1]):
                            #computes the part of the miRNA binding to the switch by finding the and region within the miRNA's corresponding andgate, then taking the corresponding number of bases from the miRNA
                            print(orderedStrands[i+1].find(j))
                            u = strand[0:len(strand) - orderedStrands[i+1].find(j)]
                            # toeholdRegion.append(u)
                            print(u)
                            thsUnpaired.append(u)
                            print(thsUnpaired)
                        else:
                            continue 
                    # toeholdRegion.append(strand[0:(.find())])
                elif (i == len(orderedStrands) - 1):
                    print("last miRNA")
                    for j in andRegions:
                        if j in (orderedStrands[i-1]):
                            print(reverse(orderedStrands[i-1]).find(j))
                            u = strand[reverse(orderedStrands[i-1]).find(j):len(orderedStrands[i-1])+3]
                            print(u)
                            thsUnpaired.append(u)
                            print(thsUnpaired)
                        else:
                            continue 
                else:
                    print("middle miRNA")
                    for j in andRegions:
                        if j in (orderedStrands[i-1]):
                            u1 = reverse(orderedStrands[i-1]).find(j)
                            print("u1: ", u1) 
                        else:
                            continue
                            
                    for j in andRegions:
                        if j in (orderedStrands[i+1]):
                            u2 = orderedStrands[i+1].find(j)
                            print("u2: ", u2)
                        else:
                            continue 
                            
                    print("start, end: ", u1, " - ", u2)
                    u3 = strand[u1:-u2]
                    print(u3)
                    thsUnpaired.append(u3)
                    
                print(thsUnpaired)
                print(thsAnds)
        
        #reorders the ang regions used and miRNA binding regions as per listAddition            
        toeholdRegion = (listAddition(thsAnds, thsUnpaired))
        
        print(toeholdRegion)
        toeholdRegion = "".join(toeholdRegion)
        print("toehold: ", toeholdRegion)
        
        #length of the unpaired binding region of the thsS
        toehold = 15
        thsDuplex = len(toeholdRegion) - toehold
        
        #thsDuplex is the closed/unzipped part of the switch
        print("thsDUplex: ", thsDuplex)
        
        print(toehold, thsDuplex)
        
        if (thsDuplex % 3 == 1):
            thsDuplex = thsDuplex - 1
            toehold +=1
        elif (thsDuplex % 3 ==2):
            thsDuplex +=1
            toehold = toehold -1
            
        print(toehold, thsDuplex)

    #sequence of the toehold switch
    toeholdSwitchSequence = (compStrand(toeholdRegion) + "GGAUUUGCAAAAAAAAGAGGAGAGUAAAAUG" + reverse(toeholdRegion)[0:thsDuplex] + "AACCUGGCGGCAGCGCAAAAG")
    print(toeholdSwitchSequence)
    
    #the entire switch structure in DU+ including the hairpin loop
    thsStructure = ("U"+ str(toehold) + "D"+ str(thsDuplex) + "(U3D5(U15)U3)U21")
    thsStrucDPP = "."*int(toehold) + "("*int(thsDuplex) + "...(((((...............)))))..." + ")"*int(thsDuplex) + "."*21
    print(str(thsStrucDPP))
    
    bindingSites = ""
    AUGs = ""
    RBSs = ""
    
    # print(toeholdSwitchSequence[0:int(toehold)])
    # print(toeholdSwitchSequence[(int(toehold) + int(thsDuplex) + 28):(int(toehold) + int(thsDuplex) + 31)])
    # print(toeholdSwitchSequence[(int(toehold) + int(thsDuplex) + 8):(int(toehold) + int(thsDuplex) + 23)])
    
    #creating a strand object for the switch so tube analysis and boltzmann sampling can be performed
    thsStrand = Strand(str(toeholdSwitchSequence), name="toeholdSwitch")
    
    #taking boltzmann samples to calculate RBS, AUG and toehold site exposure percentages
    thsStrandSamples = sample(strands = [thsStrand], num_sample = 50, model = thsModel)
    
    for struc in list(thsStrandSamples):
        print(struc)
        
#         print(str(struc)[0:int(toehold)])
#         print(str(struc)[(int(toehold) + int(thsDuplex) + 28):(int(toehold) + int(thsDuplex) + 31)])
#         print(str(struc)[(int(toehold) + int(thsDuplex) + 8):(int(toehold) + int(thsDuplex) + 23)])
        
        #computing where RBS, AUG and toehold are located
        bindingSites += (str(struc)[0:int(toehold)])
        AUGs += (str(struc)[(int(toehold) + int(thsDuplex) + 28):(int(toehold) + int(thsDuplex) + 31)])
        RBSs += (str(struc)[(int(toehold) + int(thsDuplex) + 8):(int(toehold) + int(thsDuplex) + 23)])
        
        print("Binding Sites: ", bindingSites)
        print("Start Codons: ", AUGs)
        print("Ribosome Binding SItes: ", RBSs)
    
    #calculating the unbound percentages as an average of all such values of each boltzmann sampled structure
    toeholdExpMin = 70
    toeholdExp = (1 - (bindingSites.count("(") + bindingSites.count(")"))/len(bindingSites))*100
    AUGExp = (1 - (AUGs.count("(") + AUGs.count(")"))/len(AUGs))*100
    RBSExp = (1 - (RBSs.count("(") + RBSs.count(")"))/len(RBSs))*100
    print("Toehold binding site exposure percentage: ", toeholdExp, "%")
    print("Start codon exposure percentage: ", AUGExp, "%")
    print("Ribosome binding site exposure percentage: ", RBSExp, "%")
    print(miRNADict)
    print(list(iTools.permutations(miRNADict)))
    if (toeholdExp < toeholdExpMin):
        print(list(iTools.permutations(miRNADict))[iterCount])
        # miRNADict = list(iTools.permutations(miRNADict))[iterCount]
        # iterCount += 1
        # print(iterCount)
    else:
        print("good")

    #performing tube analysis of ths strand to get an MFE structure, NOT ALWAYS REPRESENTATIVE OF RBS, AUG AND TOEHOLD UNBOUND PERCENTAGES
    t2 = Tube(strands={thsStrand:1e-5}, complexes=SetSpec(max_size=1), name="thsTube")
    t2Result = tube_analysis(tubes=[t2], compute=["pairs", "mfe"], model = thsModel)
    print(t2Result)
    thsMFE = t2Result["(toeholdSwitch)"]
    print('\nMFE proxy structure(s) ths:')
    for i, s in enumerate(thsMFE.mfe):
        print('    %2d: %s (%.2f kcal/mol)' % (i, s.structure, s.energy))
    
    
    #writing the text file for ths
    outputFile2.write(f'# toehold switch structure in DU+ notation \n')
    outputFile2.write(f'structure toeholdSwitch = ' + thsStructure + '\n')
    outputFile2.write(f'\n# base sequences for each section of the toehold switch \n')
    outputFile2.write(f'domain a = ' + compStrand(toeholdRegion) + '\n')
    outputFile2.write(f'domain b = GGAUUUGCAAAAAAAAGAGGAGA N5 AUG \n')
    outputFile2.write(f'domain c = ' + reverse(toeholdRegion)[0:thsDuplex] + '\n')
    outputFile2.write(f'domain d = AACCUGGCGGCAGCGCAAAAG \n')
    outputFile2.write(f'# toeholdSwitch sequence = ' + toeholdSwitchSequence + '\n')
    outputFile2.write(f'\n# mapping RNA domains to the toehold switch \n')
    outputFile2.write(f'toeholdSwitch.seq = a b c d')

#     #creating separate Strand object for toehold switch
#     thsObject = Strand(toeholdSwitchSequence, domains[len(domainsUsed)])
   
# comverts DU+ to dot parens plus to be passed to structDiff
def DUPlusToDPPlus(computedComplexStructure):
    global DPPlus
    DPPlus = ""
    for i in computedComplexStructure:
        if(i[0] == "U"):
            num = int(i[1:len(i)])
            DPPlus += "."*num
            print(DPPlus)
        else:
            num = int(i[1:-1])
            DPPlus += "("*num
            DPPlus += "+"
            DPPlus += ")"*num
            print(DPPlus)
           
#I'm aware nupack has their own version of this function, but it only operates on strand objects, file containing this is the analysis file in src [I think]
def structDiff(countList, struct, DPPlus):
    counter = 0
    for a, b in zip(struct, DPPlus):
        if a != b:
            counter += 1
   
    countList.append(counter)
    return counter

def tubeDef(numMiRNA):
   
    countList = []
   
    #attribute dict is a dictionary pairing each strand to a corresponding letter, these do not need to be the same letters used in the text files as these are for analysing the MFE of complexes
    attributeDict = {}

    #strand dict is the dictionary pairing each strand object to a corresponding concentration in M to then be passed onto a tube
    strandDict = {}

    #note to self:
    # strand_n = Strand("ACGUAGCUAGCUAGCAUC", name="n")
    # strand_n = Strand(Sequence, name="")


    #passing attribute dict to the Strand class' object constructor and outputting that to a strand dict
    for strand, letter in zip(strands, domainsUsed):
        attributeDict[strand] = letter
        name = letter
        strandDict[Strand(strand, name = name)] = 5e-6

    t1 = Tube(strands = strandDict, complexes = SetSpec(max_size = len(strands)), name = "tube 1")
    # tube_result = tube_analysis(tubes = [t1], compute = ["pairs", "mfe"], model = thsModel)
   
    if(numMiRNA != 1):
       
        #reordering strand to match seq to be passed to boltzmann sampler
        midpt = int((len(domainsUsed) + 1)/2)
        list1 = list(strands[midpt - 1:])
        list2 = list(strands[0:midpt - 1])
        del list1[-1]
        mergedList = []
        for i in range(len(list1)):
            mergedList += [list1[i], list2[i]]
        mergedList.append(strands[-1])
       
        global sampledStructures
        sampledStructures = sample(strands = mergedList, num_sample = 50, model = thsModel)
   
    for i in range(len(sampledStructures)):
        structDiff(countList, str(sampledStructures[i]), DPPlus)
    print(countList)
    print(str(sampledStructures))
    
    weightList = []
    for i in range(len(sampledStructures)):
        weightList.append(60)
   
    for i in range(len(sampledStructures)):
        indices = [[i-1, i+1] for i, x in enumerate(str(sampledStructures[i])) if x == "+"]
        if (str(sampledStructures[i]).startswith("(") or str(sampledStructures[i]).endswith(")")):
                weightList[i] += 10
        print(indices)
        for j in indices:              
            if ((str(sampledStructures[i]))[j[0]] == "(" and (str(sampledStructures[i]))[j[-1]] == ")"):
                weightList[i] += 5
                print(weightList)
            else:
                weightList[i] += 15
                print(weightList)
   
    scoreList = []
   
    for a, b in zip(weightList, countList):
        scoreList.append(a*b)
        print(scoreList)
        print(countList)
        print((1 - (sum(countList) / len(countList))/len(DPPlus))*100)
           
   
    print(str(sampledStructures[scoreList.index(min(scoreList))]))
    print(min(scoreList))
    
        
#Handles user input
def start():
    print("Input the number of miRNA strands")
    numMiRNA = int(input())
    if (numMiRNA == 1):
        miRNA = input()#User input must be in the same form as this: name,base sequence. Base sequence needs to be in all caps
        index = miRNA.split(",")
        if(checkCorrectCharacters(baseList, index[1])):
            index[1] = index[1].replace(" ","")
            miRNADict.update({index[0]: index[1]})
        else:
            print("Incorrect input")
        print(compStrand(list(miRNADict.values())[0]))
       
        # if (unpairedStartLength % 3 == 2):
        #     unpairedStartLength = unpairedStartLength + 1
        # elif (unpairedStartLength % 3 == 1):
        #     unpairedStartLength = unpairedStartLength - 1
        # else:
        #     unpairedStartLength = unpairedStartLength
       
        outputFile2.write(f'# toehold switch structure in DU+ notation \n')
        outputFile2.write(f'structure toeholdSwitch = ' + ' U6' + 'D' + str(len(index[1]) - 6) + '(U3D5(U15)U3)U21' + '\n')
        outputFile2.write(f'\n# base sequences for each section of the toehold switch \n')
        outputFile2.write(f'domain a = ' + compStrand(miRNA) + '\n')
        outputFile2.write(f'domain b = GGAUUUGCAAAAAAAAGAGGAGA N5 AUG \n')
        outputFile2.write(f'domain c = ' + reverse(miRNA)[0:(len(index[1]) - 6)] + '\n')
        outputFile2.write(f'domain d = AACCUGGCGGCAGCGCAAAAG \n')
        # outputFile2.write(f'# toeholdSwitch sequence = ' + toeholdSwitchSequence + '\n')
        outputFile2.write(f'\n# mapping RNA domains to the toehold switch \n')
        outputFile2.write(f'toeholdSwitch.seq = a b c d')
       
    else:
        i = 0
        while(i < numMiRNA):
            print("input the miRNA and its sequence")
            miRNA = input()#User input must be in the same form as this: name,base sequence. Base sequence needs to be in all caps
            index = miRNA.split(",")
            if(checkCorrectCharacters(baseList, index[1])):
                index[1] = index[1].replace(" ","")
                miRNADict.update({index[0]: index[1]})
                i+=1
            else:
                print("Incorrect input")
        andGateGen(miRNADict)
    unpairedParts(numMiRNA)

    #adds the base sequences of each miRNA and AND gate to strands to be used later
    for value in miRNADict.values():
        strands.append(value)

    if(numMiRNA != 1):
        domainOutputs()
        toeholdSwitchGen(numMiRNA)
        tubeDef(numMiRNA)
start()

Input the number of miRNA strands


 3


input the miRNA and its sequence


 34,UGGCAGUGUCUUAGCUGGUUGU


input the miRNA and its sequence


 373,ACUCAAAAUGGGGGCGCUUUCC


input the miRNA and its sequence


 21,CAACACCAGUCGAUGGGCUGU


STORAGE ARRAY:  ['ACCGUCA', 'ACCAACA', 'GUCUUAGC']
STORAGE ARRAY:  ['UGAGUUU', 'CGAAAGG', 'AUGGGGGC']
STORAGE ARRAY:  ['GUUGUGG', 'CCGACA', 'AGUCGAUG']
UUUUUU
CCCCCC
Done
before:  ['c', 'd', 'e']
['c', 'a', 'd', 'b', 'e']
after:  ['c', 'a', 'd', 'b', 'e']
...............
...............(((((((+)))))))
...............(((((((+)))))))......
...............(((((((+)))))))......(((((((+)))))))
...............(((((((+)))))))......(((((((+)))))))........
...............(((((((+)))))))......(((((((+)))))))........(((((((+)))))))
...............(((((((+)))))))......(((((((+)))))))........(((((((+)))))))......
...............(((((((+)))))))......(((((((+)))))))........(((((((+)))))))......(((((((+)))))))
...............(((((((+)))))))......(((((((+)))))))........(((((((+)))))))......(((((((+)))))))..............
['UGGCAGUGUCUUAGCUGGUUGU', 'ACAACCACCCCCCUUUGAGU', 'ACUCAAAAUGGGGGCGCUUUCC', 'GGAAAGCAAAAAAGGUGUUG', 'CAACACCAGUCGAUGGGCUGU']
UGGCAGUGUCUUAGCUGGUUGU  isn't an and gate
first miRNA
7
UGGC

In [26]:
compStrand("UCCUGUACUGAGCUGAAAAAAAUCCAUGCAAAAAAGGGAGCAGCUGGUGG")

'AGGACAUGACUCGACUUUUUUUAGGUACGUUUUUUCCCUCGUCGACCACC'

In [8]:
strands

['GGAAAGCAAAAAAGGUGUUG',
 'ACAGCCUUUUUUACUGCCA',
 'ACUCAAAAUGGGGGCGCUUUCC',
 'CAACACCAGUCGAUGGGCUGU',
 'UGGCAGUGUCUUAGCUGGUUGU']

In [26]:
list(miRNADict.values())

['UGGCAGUGUCUUAGCUGGUUGU', 'ACUCAAAAUGGGGGCGCUUUCC', 'CAACACCAGUCGAUGGGCUGU']